In [1]:
# Imports
import os

# import methods from FIA python script
from FIA import *

oms.LogConfigHandler().setLogLevel("DEBUG")

## Loading

In [2]:
# set path to your mzML files and workfolder
data_dir = "../../data/example data/exampleA_ecolistrains"
run_dir = "../../runs/fia_explorer"

data_dir = os.path.normpath(os.path.join(os.getcwd(), data_dir))
run_dir = os.path.normpath(os.path.join(os.getcwd(), run_dir))

# clean_dir(run_dir)

In [3]:
fia_df = load_fia_df(data_dir, ".mzXML")

Loading experiments:


100%|██████████| 6/6 [00:04<00:00,  1.49it/s]


Loading names:


100%|██████████| 6/6 [00:00<?, ?it/s]


## Data pruning & intesity accumumlation

In [4]:
# Computes mean, media or sum of binned peaks (median needs ~2* more time)
fia_df["lim_experiment"] = fia_df["experiment"].apply(lambda experiment: limit_experiment(experiment, 51, 1699, 2*10**6, statistic="sum", deepcopy=True))

#### Summing intensities

In [5]:
exp = fia_df.loc[0, "lim_experiment"]
exp.getSpectra()[0].get_peaks()[1][-50:]

array([ 61.,   0.,   0.,   0.,  58.,   0.,   0.,   3.,   0.,   0.,   0.,
         3.,   0.,   0.,   0.,   3.,   0.,   0.,   0.,   3.,   0.,   0.,
         0.,   3.,   0.,   0.,   0.,  49.,   0.,   0.,   0.,  52.,   0.,
         0.,   0.,  42.,   0.,   0.,   0., 263.,   0.,   0.,   0., 269.,
         0.,   0., 238.,   0.,   0.,   0.], dtype=float32)

In [6]:
bins = np.linspace(1, 10, 100)
np.digitize(np.array([1,1.6,2,2,3,8,8,8,8,8,8]), bins)

array([ 1,  7, 12, 12, 23, 78, 78, 78, 78, 78, 78], dtype=int64)

In [7]:
fia_df

,sample,polarity,experiment,lim_experiment
0,mg1655,-1,<pyopenms._dataframes._MSExperimentDF object a...,<pyopenms._pyopenms_5.MSExperiment object at 0...
1,mg1655,1,<pyopenms._dataframes._MSExperimentDF object a...,<pyopenms._pyopenms_5.MSExperiment object at 0...
2,nissle,-1,<pyopenms._dataframes._MSExperimentDF object a...,<pyopenms._pyopenms_5.MSExperiment object at 0...
3,nissle,1,<pyopenms._dataframes._MSExperimentDF object a...,<pyopenms._pyopenms_5.MSExperiment object at 0...
4,tue1,-1,<pyopenms._dataframes._MSExperimentDF object a...,<pyopenms._pyopenms_5.MSExperiment object at 0...
5,tue1,1,<pyopenms._dataframes._MSExperimentDF object a...,<pyopenms._pyopenms_5.MSExperiment object at 0...
